In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


In [7]:
df = pd.read_csv('training.csv')
df.columns =['Emotion','Random', 'Date', 'Query', 'User','Tweet'

,Emotion,Random,Date,Query,User,Tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [14]:
df.head()

,Emotion,Tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [8]:
df = df.drop(columns=['Random','Date','Query','User'])

In [12]:
test = pd.read_csv('testing.csv')
test.columns =['Emotion','Random', 'Date', 'Query', 'User','Tweet']
test = test.drop(columns=['Random','Date','Query','User'])

In [13]:
test.head()

,Emotion,Tweet
0,4,Reading my kindle2... Love it... Lee childs i...
1,4,"Ok, first assesment of the #kindle2 ...it fuck..."
2,4,@kenburbary You'll love your Kindle2. I've had...
3,4,@mikefish Fair enough. But i have the Kindle2...
4,4,@richardebaker no. it is too big. I'm quite ha...


In [15]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])

In [16]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final



In [17]:
df = df.dropna(subset=['Tweet'])
df['Tweet'] = df['Tweet'].apply(remove_punctuation)

In [18]:
test = test.dropna(subset=['Tweet'])
test['Tweet'] = test['Tweet'].apply(remove_punctuation)

In [20]:
X_train = df['Tweet']
y_train = df['Emotion']

In [21]:
X_test = test['Tweet']
y_test = test['Emotion']

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
V_X_train = vectorizer.fit_transform(X_train)
V_X_test = vectorizer.transform(X_test)

In [23]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [24]:
lr.fit(V_X_train,y_train)

C:\Users\queenie.d.pamatian\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [105]:
predictions = lr.predict_proba(V_X_test)


In [106]:
predictions = [i[1] for i in predictions]

In [107]:
# for pred in predictions:
 #   pred = np.round(pred)
    
predicted = []
for num in predictions:
    num = num * 4
    num = np.round(num)
    num = int(num)
    predicted.append(num)
    

In [108]:
np.unique(predicted)

array([0, 1, 2, 3, 4])

In [109]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predicted,y_test)

array([[70,  0,  2,  0,  3],
       [56,  0, 11,  0, 13],
       [26,  0, 29,  0, 29],
       [20,  0, 68,  0, 62],
       [ 5,  0, 29,  0, 74]], dtype=int64)

In [110]:
#key model metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
print("accuracy: ",accuracy_score(y_test, predicted))
print("confusion matrix: ",confusion_matrix(y_test, predicted))
#print("precision: ",precision_score(y_test,predictions))

accuracy:  0.34808853118712274
confusion matrix:  [[70 56 26 20  5]
 [ 0  0  0  0  0]
 [ 2 11 29 68 29]
 [ 0  0  0  0  0]
 [ 3 13 29 62 74]]


In [96]:
narrative = vectorizer.inverse_transform(V_X_test)

In [111]:
results = pd.DataFrame()
results['narrative'] = narrative
results['rating'] = predicted
results['orig'] = test['Tweet']
results['correct'] = test['Emotion']

In [113]:
results.tail(100)

,narrative,rating,orig,correct
397,"[epic, fail, sportsguy33, time, warner]",1,@sportsguy33 Time Warner = epic fail,0
398,"[bitly, business, china, head, hong, http, kon...",2,Lawson to head Newedge Hong Kong http//bitly/x...,2
399,"[china, guitar, house, http, in, piano, weird]",3,Weird Piano Guitar House in China http//u2sme/...,2
400,"[chevy, gm, http, photos, send, tinyurlcom, us...",3,Send us your GM/Chevy photos http//tinyurlcom/...,2
401,"[1st, caseymercier, day, gm, govt, how, hurric...",0,I know How sad is that RT @caseymercier 1st d...,0
...,...,...,...,...
492,"[1, ask, by, comment, http, indesign, latex, l...",4,Ask Programming LaTeX or InDesign submitted by...,2
493,"[all, can, come, hate, i, it, kill, latex, me,...",0,"On that note, I hate Word I hate Pages I hate ...",0
494,"[a, ahhh, back, editing, environment, i, in, l...",3,Ahhh back in a *real* text editing environment...,4
495,"[away, far, hmm, i, in, iran, see, so, trouble]",0,"Trouble in Iran, I see Hmm Iran Iran so far aw...",0
